# Training

This notebook is used to train the LSTM-based action recognition model.

Author: Lim Yun Feng, Ting Yi Xuan, Chua Sheen Wey
Last Modified: 28/10/2023

Reference : https://github.com/nam157/human_activity_recognition-/tree/main

In [ ]:
pip install scikit-multilearn

In [ ]:
map = {'A019': "Waving",
       'A020': "Kicking",
       'A042': 'Punching',
       "A147": "Running",
       "A152": "Walking"}

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard
import os
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import tensorflow as tf
import datetime

In [ ]:
# Run this only if using Google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# TensorBoard callback that will be used during the training of a Keras model.
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
# Retrieve the actions (their codenames) used in the dataset
actions = os.listdir('drive/MyDrive/UAV Human-Action Videos/')
actions = np.array(actions)
actions

array(['A019', 'A020', 'A042', 'A147', 'A152'], dtype='<U4')

In [ ]:
# Retrieve the key points of the dataset and the actions associated with them
X = np.load("/content/drive/MyDrive/FIT3162_FYP/Training/X2.npy")
y = np.load("/content/drive/MyDrive/FIT3162_FYP/Training/y2.npy")


# Split the dataset into training and testing sets
from skmultilearn.model_selection import iterative_train_test_split
X_train,y_train,X_test,y_test = iterative_train_test_split(X, y, test_size = 0.2)

In [ ]:
hehe = []
for action in ['A147', 'A042', 'A019', 'A020', 'A152']:
  hehe.append(len(os.listdir('./drive/MyDrive/New2_UAV Human-Action Videos/UAV Human-Action Videos/' + action)))

hehe2 = [0]* 5
for row in y_test:
  for i in range(5):
    hehe2[i] += row[i]
print(hehe2)
for i in range (5):
  hehe2[i] = hehe2[i] / hehe[i] * 100
hehe2, hehe

[5, 29, 26, 23, 5]


([19.230769230769234,
  20.863309352517987,
  20.0,
  20.353982300884958,
  21.73913043478261],
 [26, 139, 130, 113, 23])

In [ ]:
actions = np.array(['A147', 'A042', 'A019', 'A020', 'A152'])

In [ ]:
print(X_train,X_test,y_train,y_test)

[[[ 3.94114047e-01  1.82261959e-01 -2.43104219e-01 ...  5.87389529e-01
    2.28633612e-01  8.41270685e-01]
  [ 3.93483132e-01  1.82249889e-01 -2.40509570e-01 ...  5.83690107e-01
    2.17479944e-01  8.44553769e-01]
  [ 3.92466426e-01  1.87585458e-01 -2.46341988e-01 ...  6.10785544e-01
    1.35546982e-01  8.33474696e-01]
  ...
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]]

 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 4.11029458e-01  2.88194895e-01 -1.44009873e-01 ...  6.79486752e-01
    4.26146597e-01  7.00935364e-01]
  [ 4.04302746e-01  2.89258957e-01 -1.67733282e-01 ...  6.75379515e-01
    4.02789861e-01  6.98926806e-01]
  ...
  [ 2.51748979e-01  2.3

In [ ]:
X.shape, y.shape

((435, 30, 132), (435, 5))

In [ ]:
X_train.shape, y_train.shape

((347, 30, 132), (347, 5))

In [ ]:
X_test.shape, y_test.shape

((88, 30, 132), (88, 5))

In [ ]:
# Build the LSTM model
from keras.layers import Dropout,BatchNormalization
model = Sequential()
model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(30,132)))
# model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(30,132)))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=False, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
# actions.shape[0] = 5
model.add(Dense(actions.shape[0], activation='softmax'))
# model.add(Dense(actions.shape[0], activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 30, 128)           133632    
                                                                 
 dropout_4 (Dropout)         (None, 30, 128)           0         
                                                                 
 lstm_7 (LSTM)               (None, 30, 256)           394240    
                                                                 
 dropout_5 (Dropout)         (None, 30, 256)           0         
                                                                 
 lstm_8 (LSTM)               (None, 256)               525312    
                                                                 
 batch_normalization_2 (Bat  (None, 256)               1024      
 chNormalization)                                                
                                                      

In [ ]:
# Compile the model
model.compile(optimizer='Adam', 
              loss='categorical_crossentropy', 
              metrics=['categorical_accuracy'])

In [ ]:
# Set the early stopping monitor so the model stops training when it won't improve anymore
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor = 'val_loss', patience = 40, restore_best_weights = True)

In [ ]:
model.fit(X_train, y_train,validation_data=(X_test,y_test), epochs=2000, callbacks=[tensorboard_callback,early_stop],batch_size=64)

Epoch 1/2000
6/6 [==============================] - 11s 582ms/step - loss: 1.4423 - categorical_accuracy: 0.3602 - val_loss: 1.4193 - val_categorical_accuracy: 0.3182
Epoch 2/2000
6/6 [==============================] - 1s 223ms/step - loss: 1.1912 - categorical_accuracy: 0.4755 - val_loss: 1.4802 - val_categorical_accuracy: 0.3409
Epoch 3/2000
6/6 [==============================] - 1s 226ms/step - loss: 0.9462 - categorical_accuracy: 0.6225 - val_loss: 1.6792 - val_categorical_accuracy: 0.4205
Epoch 4/2000
6/6 [==============================] - 1s 223ms/step - loss: 0.8380 - categorical_accuracy: 0.6455 - val_loss: 1.3076 - val_categorical_accuracy: 0.4886
Epoch 5/2000
6/6 [==============================] - 2s 326ms/step - loss: 0.8131 - categorical_accuracy: 0.6859 - val_loss: 1.2789 - val_categorical_accuracy: 0.3977
Epoch 6/2000
6/6 [==============================] - 4s 762ms/step - loss: 0.7198 - categorical_accuracy: 0.7291 - val_loss: 1.1762 - val_categorical_accuracy: 0.5341
Epo

In [ ]:
# model_evaluation_history = model.evaluate()

In [ ]:
# Predict the test set results
result = model.predict(X_test)
result

3/3 [==============================] - 1s 53ms/step


array([[2.47583568e-01, 1.58892602e-01, 8.98559988e-02, 2.43179440e-01,
        2.60488391e-01],
       [1.67922392e-01, 4.94018137e-01, 1.66805968e-01, 8.98166150e-02,
        8.14369470e-02],
       [2.15569153e-01, 2.11620539e-01, 1.08759969e-01, 2.64167398e-01,
        1.99882925e-01],
       [2.38245949e-01, 2.66667336e-01, 1.31044924e-01, 2.06033200e-01,
        1.58008590e-01],
       [1.74151808e-01, 2.73180366e-01, 3.10103744e-01, 1.65708438e-01,
        7.68556595e-02],
       [1.73674330e-01, 4.04429942e-01, 1.65988162e-01, 1.50588155e-01,
        1.05319433e-01],
       [7.71605521e-02, 6.36012912e-01, 1.99907631e-01, 5.64615987e-02,
        3.04573048e-02],
       [2.98249811e-01, 1.69792160e-01, 1.00053616e-01, 2.11344361e-01,
        2.20560059e-01],
       [1.00264184e-01, 5.49372971e-01, 1.74945027e-01, 1.14214435e-01,
        6.12033717e-02],
       [1.24145508e-01, 4.81905103e-01, 1.77907631e-01, 1.42925009e-01,
        7.31167868e-02],
       [6.46137968e-02, 6.3969

In [ ]:
#  Tests the model on a random sample from the test set
actions[np.argmax(result[50])]

'A020'

In [ ]:
actions[np.argmax(y_test[50])]

'A019'

In [ ]:
# Evaluate the model
from sklearn.metrics import multilabel_confusion_matrix
y_hat = model.predict(X_test)
y_true = np.argmax(y_test, axis=1).tolist()
y_hat = np.argmax(y_hat, axis=1).tolist()
multilabel_confusion_matrix(y_true, y_hat)

3/3 [==============================] - 0s 32ms/step


array([[[81,  2],
        [ 5,  0]],

       [[45, 14],
        [ 2, 27]],

       [[61,  1],
        [10, 16]],

       [[60,  5],
        [ 4, 19]],

       [[80,  3],
        [ 4,  1]]])

In [ ]:
# Save the model
model.save("drive/MyDrive/FIT3162_FYP/Training/sw_action5.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
with open('./drive/MyDrive/FIT3162_FYP/Training/X_test.npy', 'wb') as f:
  np.save(f, X_test)

with open('./drive/MyDrive/FIT3162_FYP/Training/y_test.npy', 'wb') as f:
  np.save(f, y_test)

In [ ]:
y_pred = [np.argmax(result[i]) for i in range(len(result))]
y_true = [np.argmax(y_test[i]) for i in range(len(y_test))]

import tensorflow as tf
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score, f1_score, multilabel_confusion_matrix
balanced_acc_score = balanced_accuracy_score(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')
recall = recall_score(y_true, y_pred, average='micro')
f1 = f1_score(y_true, y_pred, average='micro')

# Print the results
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")


Precision: 0.72
Recall: 0.72
F1-score: 0.72


In [1]:
actions_name = []
for action in actions:
  actions_name.append(map[action])

from sklearn.metrics import confusion_matrix
import pandas as pd
cm = confusion_matrix(y_true, y_pred)
cm_df = pd.DataFrame(cm,
                     index = actions_name,
                     columns = actions_name
                     )
cm_df

NameError: name 'actions' is not defined

In [ ]:
# the number of videos for each action
hehe = []
for action in actions:
  hehe.append(len(os.listdir('./drive/MyDrive/New2_UAV Human-Action Videos/UAV Human-Action Videos/' + action)))
hehe

[130, 113, 139, 26, 23]

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


<IPython.core.display.Javascript object>